In [32]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
from pathlib import Path

import sisl
from sisl import viz




# Create directories for the files.

In [ ]:

structure_directory = "structures"
current_working_directory = Path()
structure_path = current_working_directory / structure_directory
structure_path.mkdir(exist_ok=True, parents=True)
structure_path

PosixPath('structures')

# Create structure

In [31]:
graphene = sisl.geom.graphene_nanoribbon(width=3, kind="zigzag", vacuum=1)
graphene.write(structure_path / "graphene.xyz") 
graphene.plot()

In [27]:
graphene.atoms

<sisl.Atoms nspecies=1, na=6, no=6>

In [ ]:
graphene += graphene
graphene.atoms


<sisl.Atoms nspecies=1, na=12, no=12>